In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%load_ext autoreload
%autoreload 2

import sys

from tqdm import tqdm



sys.path.append('../')

In [3]:
from clean_helpers import *

take_full = True
test_locally = True

# Specify here what cleaning functions you want to use
cleaning_options = ['clean_new_line', 'clean_tags', 'lowercase', \
                    'clean_punctuation', 'remove_stopwords', 'remove_numbers', 'lemmatize']


clean = {
    "clean_new_line": clean_new_line,
    "lowercase": lowercase,
    "lemmatize": lemmatize,
    "remove_stopwords": remove_stopwords,
    "translate": perform_translation,
    "clean_punctuation": clean_punctuation,
    "clean_tags" : clean_tags,
    "remove_numbers": remove_numbers,
}

NameError: name 'lemmatize' is not defined

In [10]:
input_file_pos = 'Data/train_pos.txt'
if take_full:
    input_file_pos = 'Data/train_pos_full.txt'
  
input_file_neg = 'Data/train_neg.txt'
if take_full:
    input_file_neg = 'Data/train_neg_full.txt'
    
list_of_pos_sentences = []
with open(input_file_pos, 'r') as f:
    for line in f:
        list_of_pos_sentences.append(line)
 
list_of_neg_sentences = []
with open(input_file_neg, 'r') as f:
    for line in f:
        list_of_neg_sentences.append(line)

In [8]:
from data_handling import build_sentences

df = build_sentences(list_of_pos_sentences, list_of_neg_sentences)
df.head()

for clean_option in cleaning_options:
    df = clean[clean_option](df)
    print(clean_option)
    print(df[['sentence']][:100])
    print("################################\n\n")

NameError: name 'clean' is not defined

In [11]:
if test_locally:
    shuffled = df.sample(frac=1)
    train = shuffled[:int(len(df)*0.7)]
    test = shuffled[int(len(df)*0.7)+1:]
    print(train.shape, test.shape)

    print(df.count())
    print(train[train.label == -1].count())
    print(test[test.label == 1].count())
else:
    test = []
    with open("Data/test_data.txt", 'r') as f:
        for l in f:
            id_ = l.split(",")[0]
            sentence = ",".join(l.split(",")[1:])
            test.append({
                "label": int(id_),
                "sentence": sentence
            })
    test = pd.DataFrame(test)
    test.head()
    
    for clean_option in cleaning_options:
        test = clean[clean_option](test)
        print(clean_option)
        print(test[['sentence']][:100])
        print("################################\n\n")

(1750000, 2) (749999, 2)
sentence    2500000
label       2500000
dtype: int64
sentence    875153
label       875153
dtype: int64
sentence    375153
label       375153
dtype: int64


In [16]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

def naive_bayes_model(training_data, training_labels, n):
    #construct a multinomial naive Bayes classifier based fitted on the given training data and the corresponding class labels
    classifier = Pipeline([
        ('vect', CountVectorizer(ngram_range=(1,n))), #add arguments to CountVectorizer for more efficiency
        ('tfidf', TfidfTransformer(sublinear_tf=False)),
        ('clf', MultinomialNB()),
    ])

    classifier.fit(training_data, training_labels)
    return classifier


def naive_bayes_predict(classifier, test_data):
    #predict the labels of the given test data with the given classifier
    return classifier.predict(test_data)

In [17]:
if test_locally:
    accuracies = []
    n=11

    for n in tqdm(range(1,n)):
        model = naive_bayes_model(train['sentence'], train['label'], n)
        prediction = naive_bayes_predict(model, test['sentence'])
        accuracy = np.mean(np.where(prediction==test['label'],1,0))
        accuracies.append(accuracy)
    
    plt.plot(np.arange(1,n), accuracies)
    plt.title('accuracy for naive Bayes for increasing maximum length of n-grams')
    plt.xlabel('n')
    plt.ylabel('accuracy')
    plt.show()

  0%|          | 0/10 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [19]:
model = naive_bayes_model(train['sentence'], train['label'], 1)

In [20]:
model

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [21]:
type(model)

sklearn.pipeline.Pipeline